In [1]:
# prepare some test data
from datetime import date
import app.db.database
from app.db.models import FireCentre, PlanningArea, PlanningWeatherStation, FuelType
from app.db.crud.hfi_calc import (create_fire_centre_prep_period, create_planning_area_selection_override, create_planning_area_selection_override_for_day)

with app.db.database.get_write_session_scope() as session:
    fire_centre = session.query(FireCentre).filter(FireCentre.name == 'Coastal Fire Centre').first()
    planning_area = session.query(PlanningArea).filter(PlanningArea.fire_centre_id == fire_centre.id).first()
    station = session.query(PlanningWeatherStation).filter(PlanningWeatherStation.planning_area_id==planning_area.id).first()
    fuel_type = session.query(FuelType).first()

    create_fire_centre_prep_period(session, fire_centre.id, date(2022, 2, 1), date(2022, 2, 6))
    create_planning_area_selection_override(session, planning_area.id, station.id, fuel_type.id, True)
    create_planning_area_selection_override_for_day(session, planning_area.id, date(2022,2,2), 1, 2)


In [2]:
import app.db.database
from app.db.models.hfi_calc import FireCentre
from app.db.crud.hfi_calc import get_planning_area_overrides_for_day_in_period, get_most_recent_fire_centre_prep_period, get_fire_centre_planning_area_selection_overrides



with app.db.database.get_write_session_scope() as session:
    # starting point is going to be some default
    fire_centre = session.query(FireCentre).filter(FireCentre.name == 'Coastal Fire Centre').first()

    # get a singular result (or none)
    prep_period = get_most_recent_fire_centre_prep_period(session, fire_centre.id)
    print(f'prep period: {prep_period.prep_start_day} to {prep_period.prep_end_day}')
    # get a list (n>=0)
    planning_area_overrides = get_fire_centre_planning_area_selection_overrides(session, fire_centre.id)
    print('station overrides for planning areas:')
    for override, fuel_type in planning_area_overrides:
        print(f'planning_area_id: {override.planning_area_id}, fuel: {fuel_type.abbrev} selected:{override.station_selected}')
    # get another list (n>=0)
    planning_area_overrides_for_day = get_planning_area_overrides_for_day_in_period(session, prep_period.prep_start_day, prep_period.prep_end_day)
    print('planning area overrides for period:')
    for override in planning_area_overrides_for_day:
        print(f'planning_area_id: {override.planning_area_id}, day: {override.day}, fire_starts: {override.fire_starts_min} - {override.fire_starts_max}')
    # get_planning_area_overrides_for_day(session, 1)



prep period: 2022-02-01 to 2022-02-06
station overrides for planning areas:
planning_area_id: 18, fuel: C1 selected:True
planning area overrides for period:
planning_area_id: 18, day: 2022-02-02, fire_starts: 1 - 2
